In [2]:
import pandas as pd
import os
import numpy as np
import sys
sys.path.append('..')

In [54]:
#os.chdir(r'C:\Users\Administrator\Desktop\raphi_other\repositories\template_project\src')

In [11]:
data = pd.read_csv(r'../data/raw/data/scout_data/Case_Study_Data.csv', delimiter=';')

In [13]:
#from eda_first import reduce_mem_usage
#data = reduce_mem_usage(data)

In [15]:
data.columns

Index(['article_id', 'product_tier', 'make_name', 'price', 'first_zip_digit',
       'first_registration_year', 'created_date', 'deleted_date',
       'search_views', 'detail_views', 'stock_days', 'ctr'],
      dtype='object')

In [17]:
data.shape

(78321, 12)

In [19]:
data.head()

,article_id,product_tier,make_name,price,first_zip_digit,first_registration_year,created_date,deleted_date,search_views,detail_views,stock_days,ctr
0,350625839,Basic,Mitsubishi,16750,5,2013,24.07.18,24.08.18,3091.0,123.0,30,0.03780329990294403
1,354412280,Basic,Mercedes-Benz,35950,4,2015,16.08.18,07.10.18,3283.0,223.0,52,0.06792567773378008
2,349572992,Basic,Mercedes-Benz,11950,3,1998,16.07.18,05.09.18,3247.0,265.0,51,0.0816137973514013
3,350266763,Basic,Ford,1750,6,2003,20.07.18,29.10.18,1856.0,26.0,101,0.014008620689655173
4,355688985,Basic,Mercedes-Benz,26500,3,2014,28.08.18,08.09.18,490.0,20.0,12,0.04081632653061224


In [66]:
pd.set_option('display.float_format', '{:.4f}'.format)

In [21]:
data.describe()

,article_id,price,first_zip_digit,first_registration_year,search_views,detail_views,stock_days
count,7.832100e+04,78321.000000,78321.000000,78321.000000,78311.000000,78311.000000,78321.000000
mean,3.574869e+08,15067.655316,4.632117,2011.089465,2297.502522,93.469870,35.986262
std,5.076572e+06,16374.497606,2.354275,6.538395,6339.034423,228.025588,32.214835
min,3.472324e+08,1.000000,1.000000,1924.000000,0.000000,0.000000,-3.000000
25%,3.536387e+08,5750.000000,3.000000,2008.000000,367.000000,13.000000,10.000000
50%,3.585480e+08,10900.000000,5.000000,2013.000000,920.000000,36.000000,25.000000
75%,3.614817e+08,18890.000000,7.000000,2015.000000,2234.000000,94.000000,55.000000
max,3.647040e+08,249888.000000,9.000000,2106.000000,608754.000000,13926.000000,127.000000


In [26]:
pct = np.percentile(data.loc[:, 'search_views'].fillna(np.mean(data.loc[:, 'search_views'])), 95)
print(pct)
print(data.shape)
data = data.loc[data.loc[:, 'search_views'] < pct]
print(data.shape)

7915.0
(78321, 12)
(74394, 12)


In [28]:
pct = np.percentile(data.loc[:, 'detail_views'].fillna(np.mean(data.loc[:, 'detail_views'])), 95)
print(pct)
print(data.shape)
data = data.loc[data.loc[:, 'detail_views'] < pct]
print(data.shape)

224.0
(74394, 12)
(70624, 12)


# DROP COLUMNS WHICH DONT MAKE SENSE!

In [ ]:
#drop all cars with first_registration_year of more than 2022
data = data[data['first_registration_year'] <= 2022]

#drop all columns with negative live days. (since you cannot have a listing deleted before it is created)
data = data[data['live_days'] >= 0]

#drop all negative stock days
data = data[data['stock_days'] >= 0]

#drop all cars with price less than 100 euros
data = data[data.price > 100]

In [30]:
data.columns.isnull()

array([False, False, False, False, False, False, False, False, False,
       False, False, False])

In [32]:
from src.eda_first import summarize_dataframe
summarize_dataframe(data)

,Column,Data Type,Unique Values,Missing Values,Sample Unique Values
0,article_id,int64,70619,0,"[350625839, 354412280, 350266763, 355688985, 3..."
1,product_tier,object,3,0,"[Basic, Premium, Plus]"
2,make_name,object,82,0,"[Mitsubishi, Mercedes-Benz, Ford, Volkswagen, ..."
3,price,int64,5528,0,"[16750, 35950, 1750, 26500, 24890]"
4,first_zip_digit,int64,9,0,"[5, 4, 6, 3, 7]"
5,first_registration_year,int64,85,0,"[2013, 2015, 2003, 2014, 2017]"
6,created_date,object,35,0,"[24.07.18, 16.08.18, 20.07.18, 28.08.18, 20.10..."
7,deleted_date,object,273,0,"[24.08.18, 07.10.18, 29.10.18, 08.09.18, 22.10..."
8,search_views,float64,6010,0,"[3091.0, 3283.0, 1856.0, 490.0, 125.0]"
9,detail_views,float64,224,0,"[123.0, 223.0, 26.0, 20.0, 6.0]"


In [35]:
data.to_csv('../data/interim/cars_data.csv', index=False)